In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.model_selection import RandomizedSearchCV
import joblib
import os

In [2]:
#functions to predict and check accuray
def predictAndGetAccuracyScore(model, x_valid, y_valid):
    y_pred = model.predict(x_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    return accuracy


In [3]:
#functions to execute randomized search
def getRandomizedSearch(model, param_grid, x_train, y_train):
    random_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy',
                              n_iter=2, random_state=42, verbose=2, n_jobs=-1)
    random_search.fit(x_train, y_train)
    print("Best Hyperparameters:", random_search.best_params_)
    return random_search


In [4]:
train_path = './../data/layer 7/layer_7_train.csv'
valid_path = './../data/layer 7/layer_7_valid.csv'
test_path = './../data/layer 7/layer_7_test.csv'


train_data = pd.read_csv(train_path)
valid_data = pd.read_csv(valid_path)
test_data = pd.read_csv(test_path)


In [5]:
X_train = train_data.drop(columns=["label_1", "label_2", "label_3", "label_4"])
y_test_train = train_data["label_1"]

X_valid = valid_data.drop(columns=["label_1", "label_2", "label_3", "label_4"])
Y_valid = valid_data["label_1"]

X_test = valid_data.drop(columns=["label_1", "label_2", "label_3", "label_4"])
Y_test = valid_data["label_1"]


In [10]:
#create separate dataframes for each label
x_train = {}
x_valid = {}
y_train = {}
y_valid = {}
x_test = {}
y_test = {}
Labels = ["label_1", "label_2", "label_3", "label_4"]
Features = np.array(train_data.drop(columns=["label_1", "label_2", "label_3", "label_4"]).columns)

for label in Labels:
  tr_data = train_data[train_data["label_2"].notna()] if label == "label_2" else train_data
  vl_data = valid_data[valid_data["label_2"].notna()] if label == "label_2" else valid_data

  #standardize data
  scaler = StandardScaler()

  x_train[label] = pd.DataFrame(scaler.fit_transform(tr_data.drop(Labels, axis=1)), columns=Features)
  joblib.dump(scaler, f'./../saved_components/layer7/{label}/scaler.pkl')
  y_train[label] = tr_data[label]
  x_valid[label] = pd.DataFrame(scaler.transform(vl_data.drop(Labels, axis=1)), columns=Features)
  y_valid[label] = vl_data[label]
  x_test[label] = pd.DataFrame(scaler.transform(test_data.drop(Labels, axis=1)), columns=Features)


<h4>Label 1</h4>

In [ ]:
###### Initial accuracy with svm_model
svm_model_1 = SVC(kernel="linear")
svm_model_1.fit(x_train["label_1"], y_train["label_1"])
accuracy_label_1 = predictAndGetAccuracyScore(svm_model_1, x_valid["label_1"], y_valid["label_1"])


In [11]:
#Apply PCA -Feature Extraction
pca_label_1 = PCA(n_components=0.99, svd_solver="full")
pca_label_1.fit(x_train["label_1"])
x_train_label_1_PCA = pd.DataFrame(pca_label_1.transform(x_train["label_1"]))
print(x_train_label_1_PCA.shape)
x_valid_label_1_PCA = pd.DataFrame(pca_label_1.transform(x_valid["label_1"]))

joblib.dump(pca_label_1, './../saved_components/layer7/label_1/pca.pkl')

(28520, 578)


['./../saved_components/layer7/label_1/pca.pkl']

In [ ]:
svm_model_1.fit(x_train_label_1_PCA, y_train["label_1"])
acc_pca = predictAndGetAccuracyScore(svm_model_1, x_valid_label_1_PCA, y_valid["label_1"])

In [ ]:
#Hyper parameter tuning
#Randomized search
param_dist = {
    'C': [100, 10, 1],  # Continuous uniform distribution for 'C'
    'kernel': ['rbf', 'linear'],  # Categorical distribution for 'kernel'
    'gamma': [0.001, 0.01, 0.1],
    # Add more hyperparameters and their distributions as needed
}

# Create a RandomizedSearchCV object
random_search_label_1 = getRandomizedSearch(svm_model_1, param_dist, x_train_label_1_PCA, y_train["label_1"])

# Get the best hyperparameters and the corresponding model
best_model_label_1 = random_search_label_1.best_estimator_


In [ ]:
# Evaluate the best model on validation set
acc_best_model_label_1 = best_model_label_1.score(x_valid_label_1_PCA, y_valid["label_1"])

print("Validation Accuracy with Best Model:", acc_best_model_label_1)


In [12]:
#Check accuracy with tuned model on validation set
svm_model_1_tuned = SVC(kernel="rbf", C=100, gamma=0.001)
svm_model_1_tuned.fit(x_train_label_1_PCA, y_train["label_1"])
accuracy_label_1 = predictAndGetAccuracyScore(svm_model_1_tuned, x_valid_label_1_PCA, y_valid["label_1"])


Accuracy: 98.27%


In [13]:
#Predict on test set with tuned model and PCA
y_pred_label_1 = svm_model_1_tuned.predict(pca_label_1.transform(x_test["label_1"]))
joblib.dump(svm_model_1_tuned, './../saved_components/layer7/label_1/model.pkl')

['./../saved_components/layer7/label_1/model.pkl']

In [ ]:
result_df = pd.DataFrame({'ID': test_data['ID'], 'label_1': y_pred_label_1})


<h4>Label 2</h4>

In [ ]:
###### Initial accuracy with svm_model

svm_model_2 = SVC()
svm_model_2.fit(x_train["label_2"], y_train["label_2"])
acc = predictAndGetAccuracyScore(
    svm_model_2, x_valid["label_2"], y_valid["label_2"])


In [14]:
#Apply PCA - Feature Extraction
pca_label_2 = PCA(n_components=0.98, svd_solver="full")
pca_label_2.fit(x_train["label_2"])
x_train_label2_PCA = pd.DataFrame(pca_label_2.transform(x_train["label_2"]))
print(x_train_label2_PCA.shape)
x_valid_label2_PCA = pd.DataFrame(pca_label_2.transform(x_valid["label_2"]))


(28040, 480)


In [ ]:
svm_model_2.fit(x_train_label2_PCA, y_train["label_2"])
acc = predictAndGetAccuracyScore(svm_model_2, x_valid_label2_PCA, y_valid["label_2"])


In [ ]:
#Hyper parameter tuning
#Randomized search
param_dist = {
    # Continuous uniform distribution for 'C'
    'C': [100, 10, 1, 0.1],
    'kernel': ['rbf'],  # Categorical distribution for 'kernel'
    'gamma': [0.001, 0.01, 0.1, 1, ],
    # Add more hyperparameters and their distributions as needed
}
random_search = getRandomizedSearch(
    svm_model_2, param_dist, x_train_label2_PCA, y_train["label_2"])
best_model_label_2 = random_search.best_estimator_


In [15]:
##Check accuracy with tuned model on validation set without PCA
svm_model_2_tuned = SVC(kernel='rbf', C=100, gamma=0.001)
svm_model_2_tuned.fit(x_train["label_2"], y_train["label_2"])
acc = predictAndGetAccuracyScore(svm_model_2_tuned, x_valid["label_2"], y_valid["label_2"])


Accuracy: 95.24%


In [ ]:
##Check accuracy with best model on validation set with PCA
acc_best_model_label_2 = predictAndGetAccuracyScore(best_model_label_2, x_valid_label2_PCA, y_valid["label_2"])

In [16]:
#Predict on test set with tuned model without PCA
y_pred_label_2 = svm_model_2_tuned.predict(x_test["label_2"])
joblib.dump(svm_model_2_tuned, './../saved_components/layer7/label_2/model.pkl')

['./../saved_components/layer7/label_2/model.pkl']

In [ ]:
result_df['label_2'] = y_pred_label_2


<h4>Label 3</h4>

In [ ]:
###### Initial accuracy with svm_model
svm_model_3 = SVC(kernel="linear")
svm_model_3.fit(x_train["label_3"], y_train["label_3"])
acc = predictAndGetAccuracyScore(
    svm_model_3, x_valid["label_3"], y_valid["label_3"])

In [17]:
#Apply PCA - Feature Extraction
pca_label_3 = PCA(n_components=0.92, svd_solver="full")
pca_label_3.fit(x_train["label_3"])
x_train_label_3_PCA = pd.DataFrame(pca_label_3.transform(x_train["label_3"]))
print(x_train_label_3_PCA.shape)
x_valid_label_3_PCA = pd.DataFrame(pca_label_3.transform(x_valid["label_3"]))


(28520, 234)


In [ ]:
svm_model_3.fit(x_train_label_3_PCA, y_train["label_3"])
acc = predictAndGetAccuracyScore(svm_model_3, x_valid_label_3_PCA, y_valid["label_3"])

In [ ]:
#Hyper parameter tuning
#Randomized search

label_3_param_dist = {
    # Continuous uniform distribution for 'C'
    'C': [100, 10, 1, 0.1],
    'kernel': ['rbf'],  # Categorical distribution for 'kernel'
    'gamma': [0.001, 0.01, 0.1, 1],
    # Add more hyperparameters and their distributions as needed
}


In [ ]:
random_search_label_3 = getRandomizedSearch(svm_model_3, label_3_param_dist, x_train_label_3_PCA, y_train["label_3"])

In [ ]:
best_model_label_3 = random_search_label_3.best_estimator_
acc_best_model_label_3 = predictAndGetAccuracyScore(best_model_label_3, x_valid_label_3_PCA, y_valid["label_3"])

In [18]:
##Check accuracy with tuned model on validation set without PCA
svm_model_3_tuned = SVC(kernel='rbf', C=1000, gamma=0.001)
svm_model_3_tuned.fit(x_train["label_3"], y_train["label_3"])
acc = predictAndGetAccuracyScore(svm_model_3_tuned, x_valid["label_3"], y_valid["label_3"])

Accuracy: 99.73%


In [19]:
#Predict on test set with tuned model without PCA
y_pred_label_3 = svm_model_3_tuned.predict(x_test["label_3"])
joblib.dump(svm_model_3_tuned, './../saved_components/layer7/label_3/model.pkl')

['./../saved_components/layer7/label_3/model.pkl']

In [ ]:
result_df['label_3'] = y_pred_label_3

<h4>Label 4</h4>

In [ ]:
###### Initial accuracy with svm_model
svm_model_4 = SVC()
svm_model_4.fit(x_train["label_4"], y_train["label_4"])
acc = predictAndGetAccuracyScore(svm_model_4, x_valid["label_4"], y_valid["label_4"])

In [20]:
#Apply PCA - Feature Extraction
pca_label_4 = PCA(n_components=0.97, svd_solver="full")
pca_label_4.fit(x_train["label_4"])
x_train_label_4_PCA = pd.DataFrame(pca_label_4.transform(x_train["label_4"]))
print(x_train_label_4_PCA.shape)
x_valid_label_4_PCA = pd.DataFrame(pca_label_4.transform(x_valid["label_4"]))

joblib.dump(pca_label_4, './../saved_components/layer7/label_4/pca.pkl')

(28520, 413)


['./../saved_components/layer7/label_4/pca.pkl']

In [ ]:
svm_model_4.fit(x_train_label_4_PCA, y_train["label_4"])
acc = predictAndGetAccuracyScore(svm_model_4, x_valid_label_4_PCA, y_valid["label_4"])


In [ ]:
#Hyper parameter tuning
#Randomized search
label_4_param_dist = {
    # Continuous uniform distribution for 'C'
    'C': [100, 10, 1, 0.1],
    'kernel': ['rbf'],  # Categorical distribution for 'kernel'
    'gamma': [0.001, 0.01, 0.1, 1],
    # Add more hyperparameters and their distributions as needed
}


In [ ]:
random_search_label_4 = getRandomizedSearch(svm_model_4, label_4_param_dist, x_train_label_4_PCA, y_train["label_4"])

In [ ]:
##Check accuracy with best model on validation set with PCA

best_model_label_4 = random_search_label_4.best_estimator_

acc_best_model_label_4 = predictAndGetAccuracyScore(best_model_label_4, x_valid_label_4_PCA, y_valid["label_4"])

In [21]:
##Check accuracy with tuned model on validation set with PCA
svm_model_4_tuned = SVC(kernel='rbf', C=100, gamma=0.001)
svm_model_4_tuned.fit(x_train_label_4_PCA, y_train["label_4"])
acc = predictAndGetAccuracyScore(svm_model_4_tuned, x_valid_label_4_PCA, y_valid["label_4"])


Accuracy: 97.07%


In [22]:
#Predict on test set with tuned model with PCA
y_pred_label_4 = svm_model_4_tuned.predict(pca_label_4.transform(x_test["label_4"]))
joblib.dump(svm_model_4_tuned, './../saved_components/layer7/label_4/model.pkl')

['./../saved_components/layer7/label_4/model.pkl']

In [ ]:
result_df['label_4'] = y_pred_label_4

In [ ]:
result_df.to_csv("solutions.csv", index=False)